> ## Import Package

In [1]:
import os
import torch
import numpy as np
import pandas as pd
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

> ## Define Functions
> **encode_from_utf8**: returns a list of bit that converted from character represented by UTF-8<br>
> **decode_from_bin**: returns a character represented by UTF-8 that converted from list of bit<br>

In [2]:
def encode_from_utf8(x):
    return list(map(float,bin(int(x.encode().hex(),16))[2:]))

In [3]:
def decode_from_bin(x):
    res = ""
    for i in x:
        res+=str(round(i))
        pass
    return bytearray.fromhex(hex(int(res, 2))[2:]).decode()

In [4]:
def decode_from_bin2(x):
    res = ""
    for i in x:
        res+=str(int(i>0))
        pass
    return bytearray.fromhex(hex(int(res, 2))[2:]).decode()

> ## Define Dataset Class

In [5]:
class KoreanHandwritingDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.dataset = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        img_name = self.dataset.iloc[idx, 0]
        image = io.imread(img_name)
        label = self.dataset.iloc[idx, 1]
        label = np.array([label])
        sample = {'image': image, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample
    pass

> ## Define Transform Class

In [6]:
class ToTensor(object):
    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        image = image/255.0
        return {'image': torch.from_numpy(image.reshape(1,64,64)),
                'label': torch.tensor(encode_from_utf8(label[0]))}

> ## Define Model

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.avgpool1 = nn.AvgPool2d(2,2) # kernel size 2x2 (32 = 64/2)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(2,2) # kernel size 2x2 (32 = 64/2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(2,2) # kernel size 2x2 (16 = 32/2)
        self.linear1 = nn.Linear(1152,378)
        self.relu3 = nn.ReLU()
        self.linear2 = nn.Linear(378,128)
        self.relu4 = nn.ReLU()
        self.linear3 = nn.Linear(128,24)
        pass
    def forward(self,x):
        out = self.avgpool1(x)
        if DEBUG_TRAIN : print('avgpool1: ', out.shape)
        out = self.conv1(out)
        if DEBUG_TRAIN : print('conv1: ', out.shape)
        out = self.relu1(out)
        if DEBUG_TRAIN : print('relu1: ', out.shape)
        out = self.maxpool1(out)
        if DEBUG_TRAIN : print('maxpool1: ', out.shape)
        out = self.conv2(out)
        if DEBUG_TRAIN : print('conv2: ', out.shape)
        out = self.relu2(out)
        if DEBUG_TRAIN : print('relu2: ', out.shape)
        out = self.maxpool2(out)
        if DEBUG_TRAIN : print('maxpool2: ', out.shape)
        out = out.view(out.size(0),-1)
        if DEBUG_TRAIN : print('view: ', out.shape)
        out = self.linear1(out)
        if DEBUG_TRAIN : print('linear1: ', out.shape)
        out = self.relu3(out)
        if DEBUG_TRAIN : print('relu3: ', out.shape)
        out = self.linear2(out)
        if DEBUG_TRAIN : print('linear2: ', out.shape)
        out = self.relu4(out)
        if DEBUG_TRAIN : print('relu4: ', out.shape)
        out = self.linear3(out)
        if DEBUG_TRAIN : print('linear3: ', out.shape)
        if DEBUG_TRAIN : print()
        return out

> ## Create Model
> if saved model exist, load model from disk

In [12]:
DEBUG_TRAIN = False
DEBUG_DATA = False
TRAIN = False

MODEL_PATH = './saved_model/BCEWithLogitsLoss-New.tar'
CSV_FILE = "./image-data/labels-map.csv"
ROOT_DIR = "./image-data/hangul-images"

epoch = 0
batch_size = 512
learning_rate = 0.0001

korean_dataset = KoreanHandwritingDataset(CSV_FILE, ROOT_DIR, transform=transforms.Compose([ToTensor()]))
dataloader = DataLoader(korean_dataset, batch_size = batch_size, shuffle = True, num_workers = 0, pin_memory=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = CNN().double().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
loss_function = nn.BCEWithLogitsLoss()

if(os.path.isfile(MODEL_PATH)):
    checkpoint = torch.load(MODEL_PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    if TRAIN:
        model.train()
    else:
        model.eval()
    print('model loaded!')

model loaded!


> ## Train Model

In [13]:
if TRAIN:
    model.to(device)
    model.train()
    for i in range(100):
        for j, sample in enumerate(dataloader):
            x = sample['image'].to(device)
            y = sample['label'].to(device)
            optimizer.zero_grad()
            output = model.forward(x)
            loss = loss_function(output,y)
            loss.backward()
            optimizer.step()
            if j == 0:
                if DEBUG_DATA:
                    plt.figure(figsize=(32, 32))
                    for k, img in enumerate(x):
                        plt.subplot(8, 8, k+1)
                        plt.imshow(img.squeeze())
                        plt.xticks([])
                        plt.yticks([])
                        pass
                    plt.show()
                    pass
                pass
            pass
        epoch += 1
        print('epoch: ', epoch, 'loss: ', loss)
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'epoch': epoch,
            'loss': loss,
        }, MODEL_PATH)
        pass
    pass

> ## Test Model

In [14]:
with torch.no_grad():
    model.to(device)
    model.eval()
    scoreboard = []
    for j, sample in enumerate(dataloader):
        x = sample['image'].to(device)
        y = sample['label'].to(device)
        output = model.forward(x)
        for i in range(len(output)):
            try:
                out = decode_from_bin2(output[i].tolist())
            except UnicodeDecodeError:
                out = 'UnicodeDecodeError'
            y_ = decode_from_bin(y[i].tolist())
            #print(", 실제 값: ", y_, "출력 값: ", out)
            if out==y_:
                scoreboard.append(1)
                pass
            else:
                scoreboard.append(0)
                pass
            pass
        pass
    model.to('cpu')
    pass

In [15]:
sum=0
for i in range(len(scoreboard)):
    sum+=int(scoreboard[i])
performance=sum/len(scoreboard)*100
print("performance: ",performance,"%")

performance:  78.25901940185375 %
